In [87]:
import warnings
warnings.filterwarnings("ignore")

import os
import logging
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))


import pandas as pd


##
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import mlflow
from sklearn.model_selection import KFold

# custom functions
import sys
sys.path.append("/Users/java/Documents/10Acadamy/week - 3/Rossmann Pharmaceuticals")
from myscripts import util
from myscripts import plot

In [63]:
## Read Datas

store_file = '../data/store.csv'
test_file = '../data/test.csv'
train_file = '../data/train.csv'

In [64]:
train_df = pd.read_csv(train_file, parse_dates = True, low_memory = False)

In [4]:
test_df = pd.read_csv(test_file,parse_dates = True, low_memory = False)

### Preprocessing

In [65]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df = util.add_month_year(train_df)
# train_df.isnull().sum()

In [66]:
## Lable Date and StateHoiday Train 
lb_encode = LabelEncoder()
train_df['Date'] = lb_encode.fit_transform(train_df['Date'])
train_df['StateHoliday'] = lb_encode.fit_transform(train_df['StateHoliday'])
# train_df.info()

In [67]:
### columns to use
columns = ['DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday', 'Year', 'Month', 'Day', 'WeekOfYear']

In [76]:
sample_train = train_df.sample(1000)

train_x = sample_train[columns]
train_y = sample_train[['Sales']]

In [77]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X = x_scaler.fit_transform(train_x)
Y = y_scaler.fit_transform(train_y)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [79]:
clf = RandomForestRegressor(n_estimators = 100, max_depth=5, random_state=0)
clf.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, random_state=0)

In [81]:
score = clf.score(X_test, y_test)
print(f"Prediction Score of the Model is {round(score * 100, 2)}%")

Prediction Score of the Model is 56.45%


In [92]:
def train_model(X, Y, model_type='LinearRegression'):
    model = LinearRegression()
    
    if(model_type == 'RandomForest'):
        model = RandomForestRegressor(n_estimators = 100, max_depth=5, random_state=0)
    
        
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X):
        with mlflow.start_run(run_name=model_type, nested = True):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]

            model.fit(X_train, y_train)
            score = model.score(X_test, y_test)
            
            if(model_type == 'LinearRegression'):
                mlflow.log_param('LR-Score', score)
                mlflow.sklearn.log_model(reg, "lin_regression_models")
                
            if(model_type == 'RandomForest'):
                mlflow.log_param('RF-Score', score)

            print(f"Prediction Score of the {model_type} is {round(score * 100, 2)}%")
    print()

train_model(X, Y, "RandomForest")
train_model(X, Y)

Prediction Score of the RandomForest is 54.65%
Prediction Score of the RandomForest is 52.05%
Prediction Score of the RandomForest is 58.57%
Prediction Score of the RandomForest is 55.39%
Prediction Score of the RandomForest is 49.76%

Prediction Score of the LinearRegression is 56.3%
Prediction Score of the LinearRegression is 50.25%
Prediction Score of the LinearRegression is 59.82%
Prediction Score of the LinearRegression is 55.7%
Prediction Score of the LinearRegression is 50.23%



### Building models with sklearn pipelines

Prediction Score of the Model is 17.5%


### Choose a loss function

### Post Prediction analysis

### Serialize models

### Building model with deep learning

### Using MLFlow to serve the prediction